# Maximum Flow Algorithmen
Mit Flussgraphen lassen sich alle Probleme modellieren, bei denen ein Fluss von einer Quelle zu einer Senke stattfindet. Dazu zählen beispielsweise Rohrleitungen und Kabel, aber auch Betrachtungen des Straßenverkehrs.

In [ ]:
from tui_dsmt.graph import MaximumFlow, FordFulkerson, EdmondsKarp
from graph_data import G, G2

## Inhaltsverzeichnis
- [Flussgraphen](#Flussgraphen)
- [Ford-Fulkerson-Algorithmus](#Ford-Fulkerson-Algorithmus)
- [Komplexität durch Pfadwahl](#Komplexität-durch-Pfadwahl)
- [Edmonds-Karp-Algorithmus](#Edmonds-Karp-Algorithmus)
- [Kantengewichte und maximaler Fluss](#Kantengewichte-und-maximaler-Fluss)

## Flussgraphen
Ein **Flussgraph** - häufig auch als *Netzwerk* bezeichnet - ist ein <u>gerichteter</u> Graph $G = (V, E)$ mit zwei besonderen Knoten. $s$ bezeichnet eine Quelle (**S**ource) und $t$ eine Senke (**T**arget). Zusätzlich existiert eine Funktion $u : E \rightarrow \mathbb{R}_+$, die jeder Kante eine nichtnegative **Kapazität** zuweist.

Ein **Fluss** ist eine Funktion $f : E \rightarrow \mathbb{R}_+$, die jeder Kante einen nichtnegativen Fluss zuweist. Für den Fluss müssen zwei Bedingungen gelten:
- Der Fluss muss kleiner oder gleich der Kapazität sein: $\forall e \in E : f(e) \leq u(e)$
- Abgesehen von der Quelle und der Senke muss in jeden Knoten so viel hinein- wie hinausfließen.

Im Beispiel der Rohrleitungen könnte es sich bei den Knoten um Pumpen handeln. Währenddessen beschreibt die Kapazität den maximalen Durchfluss und der Fluss den aktuellen Durchfluss. Fluss und Kapazität werden in grafischen Darstellungen in der Regel durch einen Schrägstrich getrennt neben der jeweiligen Kante notiert.

In [ ]:
MaximumFlow(G)

## Ford-Fulkerson-Algorithmus
Der Algorithmus von Ford und Fulkerson versucht Pfade von der Quelle zur Senke und währenddessen den Fluss kontinuierlich zu erhöhen. Es sind jedoch noch weitere Konzepte notwendig, um den Algorithmus zu verstehen:
- Der **Residualgraph** besitzt die gleichen Knoten- und Kantenmengen, wird aber zusätzlich um Rückkanten erweitert. (Für jede gerichtete Kante wird eine Kante mit vertauschten Start- und Zielknoten eingefügt.)
- Ein **augmentierender Pfad** verbindet Quelle und Senke und besitzt freie Kapazitäten zur Erhöhung des Flusses.
- Ein **Flaschenhals** ist die Kante entlang eines Pfades, welche die geringste verbleibende Kapazität besitzt.

Der Algorithmus arbeitet dann wie folgt:
1. Aus dem Graphen wird der Residualgraph gebildet.
2. Jeder Kante wird der Fluss $0$ zugeordnet.
3. Solange ein Pfad von $s$ nach $t$ **im Residualgraphen** gefunden werden kann
    1. ist der Wert des Flaschenhalses zu bestimmen,
    2. für alle Kanten im Pfad zum Fluss der Wert des Flaschenhalses zu addieren und
    3. für alle Gegenkanten des Pfades vom Fluss der Wert des Flaschenhalses zu subtrahieren.

Für rationale Kapazitäten endet der Algorithmus in endlicher Zeit.

In [ ]:
FordFulkerson(G)

In den Schritten $4$ und $5$ ist im Beispiel gut zu sehen, wofür der Residualgraph mit den Rückkanten gebraucht wird. Um einen maximalen Fluss zu erreichen, muss in diesen Schritten der Fluss der Kante $a \rightarrow b$ auf $s \rightarrow b$ umgeleitet werden. Erst in den darauffolgenden Schritten $6$ und $7$ wird der Fluss entlang der Kante $a \rightarrow b$ als Teil eines anderen augmentierenden Pfades wieder gesteigert.

## Komplexität durch Pfadwahl
Sind alle Kapazitäten rational, endet der Algorithmus nach endlich vielen Schritten. Die Komplexität hängt aber maßgeblich von der Wahl des nächsten augmentierenden Pfades ab. Wird beispielsweise eine einfache Tiefensuche verwendet, ist die Reihenfolge der Speicherung in der Adjazenzliste beziehungsweise Adjazenzmatrix maßgebend.

Es lassen sich daher leicht Graphen konstruieren, die trotz kleiner Knoten- und Kantenmengen eine große Anzahl an Iterationen benötigen. Das nachfolgende Beispiel wurde so gewählt, dass der Algorithmus immer einen Pfad wählt, der die Kante $a \rightarrow b$ oder deren Rückkante enthält. Der Flaschenhals bekommt daher den Wert $1$, sodass in jeder Iteration der Fluss lediglich um diesen Wert erhöht werden kann.

In [ ]:
FordFulkerson(G2)

## Edmonds-Karp-Algorithmus
Abhilfe kann zum Beispiel geschaffen werden, indem in jedem Schritt der kürzeste augmentierende Pfad gewählt wird, um das häufige Iterieren über Rückkanten mit geringer Kapazität zu vermeiden. Der Algorithmus von Edmonds und Karp ist eine verbesserte Implementierung des Algorithmus von Ford und Fulkerson. Er verwendet eine Breitensuche zur Auswahl des nächsten augmentierenden Pfades, findet damit immer den kürzesten und bewältigt das zuvor gezeigte Beispiel damit in deutlich kürzerer Zeit.

In [ ]:
EdmondsKarp(G2)

## Kantengewichte und maximaler Fluss
Gelegentlich müssen Kantengewichte einbezogen werden. Das ist zum Beispiel der Fall, wenn die Nutzung einer Leitung abgerechnet wird und verschiedene Wege auch unterschiedliche Preise aufrufen.

Man unterteilt dann in der Regel in zwei Szenarien:
1. Die Kosten für die Verwendung einer Kante bleiben unabhängig vom tatsächlich genutzten Volumen konstant. Man könnte also von einer Art Flatrate sprechen. In diesem Fall ist eine Bevorzugung der Kanten mit niedrigem Gewicht ausreichend, um eine Lösung mit minimalen Kosten zu finden.
2. Die Kosten für die Verwendung einer Kante hängen vom tatsächlichen Durchfluss ab und verändert sich demnach im Laufe der Suche nach einer Lösung.